<a href="https://colab.research.google.com/github/T-Sawao/diveintocode-ml/blob/master/term2_sprint12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.2次元の畳み込みニューラルネットワークスクラッチ

2次元に対応した畳み込みニューラルネットワーク（CNN）のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


プーリング層なども作成することで、CNNの基本形を完成させます。クラスの名前はScratch2dCNNClassifierとしてください。


### データセットの用意  
引き続きMNISTデータセットを使用します。2次元畳み込み層へは、28×28の状態で入力します。


今回は白黒画像ですからチャンネルは1つしかありませんが、チャンネル方向の軸は用意しておく必要があります。


(n_samples, n_channels, height, width)のNCHWまたは(n_samples, height, width, n_channels)のNHWCどちらかの形にしてください。


## 【問題1】2次元畳み込み層の作成  
1次元畳み込み層のクラスConv1dを発展させ、2次元畳み込み層のクラスConv2dを作成してください。


フォワードプロパゲーションの数式は以下のようになります。
$$a_{i, j, m} = \sum_{k=0}^{K-1} \sum_{s=0}^{F_h -1}\sum_{t=0}^{F_w -1} x_{(i+s),(j+t),k}w_{s,t,k,m} + b_m$$
$a_{i,j,m}$ : 出力される配列のi行j列、mチャンネルの値

$i$ : 配列の行方向のインデックス

$j$ : 配列の列方向のインデックス

$m$ : 出力チャンネルのインデックス

$K$ : 入力チャンネル数

$F_h,F_w$ : 高さ方向（h）と幅方向（w）のフィルタのサイズ

$x_{(i+s),(j+t),k}$ : 入力の配列の(i+s)行(j+t)列、kチャンネルの値

$w_{s,t,k,m}$ : 重みの配列のs行t列目。kチャンネルの入力に対して、mチャンネルへ出力する重み

$b_m$ : mチャンネルへの出力のバイアス項

全てスカラーです。

次に更新式です。1次元畳み込み層や全結合層と同じ形です。
$$w'_{s, t, k, m} = w_{s, t, k, m} - \alpha \frac{\partial L}{\partial w_{s, t, k, m}}$$

$$b'_m = b_m - \alpha \frac{\partial L}{\partial b_m}$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial w_{s, t, k, m}}$ ： $w_{s,t,k,m}$ に関する損失 $L$ の勾配

$\frac{\partial L}{\partial b_m}$ : $b_m$ に関する損失$L$ の勾配

勾配$\frac{\partial L}{\partial w_{s, t, k, m}}$ や $\frac{\partial L}{\partial b_m}$ を求めるためのバックプロパゲーションの数式が以下である。
$$\frac{\partial L}{\partial w_{s, t, k, m}} = \sum_{i=0}^{N_{out, h}-1} \sum_{j=0}^{N_{out, w}-1} \frac{\partial L}{\partial a_{i,j,m}} x_{(i+s)(j+k),k}$$$$\frac{\partial L}{\partial b_m} =  \sum_{i=0}^{N_{out, h}-1} \sum_{j=0}^{N_{out, w}-1} \frac{\partial L}{\partial a_{i,j,m}}$$

$\frac{\partial L}{\partial a_{i,j,m}}$ : 勾配の配列のi行j列、mチャンネルの値

$N_{out,h},N_{out,w}$ : 高さ方向（h）と幅方向（w）の出力のサイズ

前の層に流す誤差の数式は以下です。
$$\frac{\partial L}{\partial x_{i,j,k}} = \sum_{m=0}^{M-1} \sum_{s=0}^{F_{h-1}} \sum_{t=0}^{F_{w-1}} \frac{\partial L}{\partial a_{(i-s)(j-t),m}}w_{s, t, k, m}$$

$\frac{\partial L}{\partial x_{i,j,k}}$ : 前の層に流す誤差の配列のi列j行、kチャンネルの値

$M$ : 出力チャンネル数
ただし、$i−s&lt;0$ または$i−s&gt;N_{out,h}−1$ または$j−t&lt;0$ または $j−t&gt;N_{out,w}−1$ のとき
$\frac{\partial L}{\partial a_{(i-s)(j-t),m}}w_{s, t, k, m}=0$です。


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as setattr
from sklearn.metrics import accuracy_score

### 1.1.1（予備知識) im2colクラスの作成

In [ ]:
class im2col():
  def __init__(self, filter_h=3, filter_w=3, stride=1, padding=0):
    self.FH = filter_h       # フィルターの高さ
    self.FW = filter_w      #　フィルターの幅
    self.str = stride        # ストライド値
    self.pad = padding     # パディング(今回未使用)

  def im2col(self, X):
    N, C, H, W = X.shape
    print('========X===========\n', X.shape)
    print('=====================')
    print('========FH===========\n', self.FH)
    print('=====================')
    print('========FW===========\n', self.FW)
    print('=====================')

    # h(上下)、w(左右)にフィルターが移動する回数
    out_h, out_w = self._output_size(H, W)
    print('========out_h===========\n', out_h)
    print('=====================')
    print('========out_w===========\n', out_w)
    print('=====================')

    #パディングの作成
    # img = np.pad(img, [(前0画像セット数, 後0画像セット数), (前0画像数, 後0画像数), (上0埋め数, 下0埋め数), (左0埋め数, 右0埋め数)], 'constant')
    # https://qiita.com/jun40vn/items/7be9f288edede284db97
    img = np.pad(X, [(0,0), (0,0), (self.pad, self.pad), (self.pad, self.pad)], 'constant')

    # 出力値を記載する枠を作成
    col = np.zeros([N, C, self.FH, self.FW, out_h, out_w])
    print('========col0===========\n', col)
    print('=====================')

    for y in range(self.FH):
      y_max = y + self.str * out_h
      for x in range(self.FW):
        x_max = x + self.str * out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:self.str, x:x_max:self.str]
                                      #-----------------
                                      #0~列の端までstride数飛ばしで習得する
    print('========col1===========\n', col)
    print('=====================')

    # transposeで配列の入れ替え（N, out_h, out_w, C, self.FH, self.FW)の順番に変更。 
    # reshapeに-1を指定することで、指定した行数になるように列数は自動調整。
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    return col

  def col2im(self, col, input_shape):
               # ----------------
               #入力データの形状(例:(10, 1, 28, 28))
    N, C, H, W = input_shape
    out_h, out_w = self._output_size(H, W)

    # transposeで配列の入れ替え（N, C, self.FH, self.FW, out_h, out_w)の順番に変更。 
    col = col.reshape(N, out_h, out_w, C, self.FH, self.FW).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*self.pad + self.str - 1, W + 2*self.pad + self.str - 1))
    for y in range(self.FH):
        y_max = y + self.str*out_h
        for x in range(self.FW):
            x_max = x + self.str*out_w
            img[:, :, y:y_max:self.str, x:x_max:self.str] += col[:, :, y, x, :, :]

    return img[:, :, self.pad:H + self.pad, self.pad:W + self.pad]

# 問題2 ----------------------------------------------
  def _output_size(self, H, W):
    out_h = int((H + 2*self.pad - self.FH) / self.str + 1)       
    out_w = int((W + 2*self.pad - self.FW) / self.str + 1)
    return out_h, out_w 

### 1.1.2 （予備知識） サンプルデータ

In [ ]:
data = np.random.rand(1, 1, 7, 7) * 100 // 1
print('========== input ==========\n', data)

### 1.1.3 （予備知識） im2colメソッドの動作確認

In [ ]:
col = im2col()
cc = col.im2col(data)
print("col.im2col(data).shape:",cc.shape)
print(cc[:5])

### 1.1.4 （予備知識） col2imメソッドの動作確認

In [ ]:
dcc = col.col2im(cc, data.shape)
print(dcc)

### 1.2.1　（解答） 畳み込み層の実装

# チャンネル数１の畳み込みネットワーク
class Conv2d:
    def __init__(self, w, b, stride=1, padding=0):
      self.w = w
      self.b = b
      self.str = stride
      self.pad = padding

      # 中間データ(backward時に使用)
      self.x = None
      self.col = None
      self.col_W = None

      # 重み・バイアスパラメーターの勾配
      self.dw = None
      self.db = None

    def forward(self, x):
      self.x = x
      FN, C, FH, FW = self.w.shape
      N, C, H, W = x.shape
      out_h = 1 + ((H + (2 * self.pad)  - FH) // self.str)
      out_w = 1 + ((W + (2 * self.pad) - FW) // self.str)
      
      # im2colクラスを定義し、実行する。
      self.col = im2col(filter_h=FH, filter_w=FW, stride=self.str, padding=self.pad)
      self.col_im = self.col.im2col(self.x)
      self.col_w = self.w.reshape(FN, -1).T
      # print('==========col===========\n', self.col_im)
      # print('=====================')
      # print('=========col_w===========\n', self.col_w)
      # print('=====================')

      out = (self.col_im @ self.col_w) + self.b
      # print('=========out1===========\n', out)
      # print('=====================')
      # reshapeで出力サイズを指定の形状に再構成、transposeで順番を入れ替え。
      out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
      # print('=========out2===========\n', out)
      # print('=====================')
      return out

    def backward(self, dout):
      FN, C, FH, FW = self.w.shape
      # print('========dout1===========\n', dout)
      # print('=====================')
      dout = dout.transpose(0,2,3,1).reshape(-1,FN)
      # print('========dout2===========\n', dout)
      # print('=====================')

      self.db = np.sum(dout, axis=0)
      # print('==========db===========\n', self.db)
      # print('=====================')
      self.dw = self.col_im.T @ dout
      # print('==========dw1===========\n', self.dw)
      # print('=====================')
      self.dw = self.dw.transpose(1, 0).reshape(FN, C, FH, FW)
      # print('=========dw2===========\n', self.dw)
      # print('=====================')
      dcol = dout @ self.col_w.T
      # print('========dcol===========\n', dcol)
      # print('=====================')
      # im2col状態を戻す処理
      dx = self.col.col2im(dcol, self.x.shape)
      # print('==========dx===========\n', dx)
      # print('=====================')
      return dx

### 1.2.2（解答）フォワードプロパゲーション

In [ ]:
x = np.array([[2, 3, 4, 5], [1, 2, 3, 4]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）
w = np.array([[[[1,1,1],
            [1,1,1]],
            [[1,1,1],
            [2,1,1]],
            [[2,1,1],
            [1,1,2]]]])
b = np.array([3, 2, 1]) # （出力チャンネル数）
b=np.array([b]*1)
display("b.shape:", b.shape)
w = w.transpose(1,0,2,3)
display("w.shape:", w.shape)
x = np.array([[x]*1]*1)
display("x.shape:", x.shape)

In [ ]:
con = Conv2d(w, b)
x_con = con.forward(x)
x_con

### 1.2.3 （解答）バックプロパゲーション

In [ ]:
dout = np.array([[[52,56]],
                 [[32,35]],
                 [[9,11]]])
dout = np.array([dout]*1)
dout.shape

In [ ]:
dout_im =con.backward(dout)
dout_im

## 【問題2】2次元畳み込み後の出力サイズ
畳み込みを行うと特徴マップのサイズが変化します。どのように変化するかは以下の数式から求められます。この計算を行う関数を作成してください。
$$N_{h, out} = \frac{N_{h, in} + 2P_h - F_h}{S_h} + 1$$$$N_{w, out} = \frac{N_{w, in} + 2P_w - F_w}{S_w} + 1$$

$N_{out}$ : 出力のサイズ（特徴量の数）

$N_{in}$ : 入力のサイズ（特徴量の数）

$P$ : ある方向へのパディングの数

$F$ : フィルタのサイズ

$S$ : ストライドのサイズ

$h$ が高さ方向、$w$ が幅方向である

### 2.1.1（解答）　 im2colクラスへ実装済み

## 【問題3】最大プーリング層の作成 最大プーリング層のクラスMaxPool2Dを作成してください。プーリング層は数式で表さない方が分かりやすい部分もありますが、数式で表すとフォワードプロパゲーションは以下のようになります。
$$a_{i,j,k} = \max_{(p,q)\in P_{i,j}}x_{p,q,k}$$

$P_{i,j}$ : i行j列への出力する場合の入力配列のインデックスの集合。 $S_h×S_w$ の範囲内の行$（p）$と列$（q）$

$S_h,S_w$ : 高さ方向$（h）$と幅方向$（w）$のストライドのサイズ

$(p,q)\in P_{i,j}$ : $P_{i,j}$ に含まれる行$（p）$と列$（q）$のインデックス

$a_{i,j,m}$ : 出力される配列のi行j列、kチャンネルの値

$x_{p,q,k}$ : 入力の配列の$p$行$q$列、$k$チャンネルの値

ある範囲の中でチャンネル方向の軸は残したまま最大値を計算することになります。

バックプロパゲーションのためには、フォワードプロパゲーションのときの最大値のインデックス $(p,q)$ を保持しておく必要があります。フォワード時に最大値を持っていた箇所にそのままの誤差を流し、そこ以外には0を入れるためです。

class Pooling:
  def __init__(self, pool_h=2, pool_w=2, stride=2, padding=0):
    self.PH = pool_h
    self.PW = pool_w
    self.str = stride
    self.pad = padding
    # im2colクラスの定義
    self.col = im2col(filter_h=self.PH, filter_w=self.PW, stride=self.str, padding=self.pad)

    self.x = None
    self.arg_max = None

  def forward(self, x):
    print('=========x===========\n', x.shape)
    print('=====================')
    N, C, H, W = x.shape
    # out_h = int(1 + (H - self.pool_h) / self.stride)
    # out_w = int(1 + (W - self.pool_w) / self.stride)
    # im2colクラスのメソッドで、フィルターの上下、左右への移動数を求める。
    out_h, out_w = self.col._output_size(self.PH, self.PW)
    print('=========out_h===========\n', out_h)
    print('=====================')
    print('=========out_w===========\n', out_w)
    print('=====================')

    # col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    # col = col.reshape(-1, self.pool_h*self.pool_w)
    # 2次元配列に整形（2x2のフィルターで区切り、各カーネルの値を一列に並べフィルターを最後まで掛ける）
    self.col_im = self.col.im2col(x)
    print('=========col1===========\n', self.col_im)
    print('=====================')
    # １行をPHxPW（４）列の配列に整形。
    col = self.col_im.reshape(-1, self.PH*self.PW)
    print('=========col2===========\n', col)
    print('=====================')

    # colの最大値のindexを返す
    arg_max = np.argmax(col, axis=1)
    print('=========max===========\n', arg_max)
    print('=====================')
    # indexから値を返す。
    out = np.max(col, axis=1)
    print('=========out1===========\n', out)
    print('=====================')
    # 配列順を変更し、次に渡したい型に整形。
    out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
    print('=========out2===========\n', out)
    print('=====================')

    self.x = x
    self.arg_max = arg_max

    return out

  def backward(self, dout):
    print('=========dout1===========\n', dout)
    print('=====================')
    dout = dout.transpose(0, 2, 3, 1)
    print('=========dout2===========\n', dout)
    print('=====================')
    
    pool_size = self.PH * self.PW
    print('=========pool_size===========\n', pool_size)
    print('=====================')
    dmax = np.zeros((dout.size, pool_size))
    print('=========dmax1===========\n', dmax)
    print('=====================')
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
    print('=========dmax2===========\n', dmax)
    print('=====================')
    dmax = dmax.reshape(dout.shape + (pool_size,)) 
    print('=========dmax3===========\n', dmax)
    print('=====================')
    
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
    print('=========dcol===========\n', dcol)
    print('=====================')
    dx = self.col.col2im(dcol, self.x.shape)
    print('=========dx===========\n', dx)
    print('=====================')
    
    return dx

In [ ]:
xin = np.array([[[[1,3,2,9],
                  [7,4,1,5],
                  [8,5,2,3],
                  [4,2,1,4]],
                 
                 [[1,3,2,9],
                  [7,4,1,5],
                  [8,5,2,3],
                  [4,2,1,4]]],
               [[[1,3,2,9],
                  [7,4,1,5],
                  [8,5,2,3],
                  [4,2,1,4]],
                 
                 [[1,3,2,9],
                  [7,4,1,5],
                  [8,5,2,3],
                  [4,2,1,4]]]])
pool=Pooling()
pool.forward(xin).shape

In [ ]:
dP = np.array([[[[1,2],
                 [3,4]],
                
                [[1,2],
                 [3,4]]],
              [[[1,2],
                 [3,4]],
                
                [[1,2],
                 [3,4]]]])
pool.backward(dP)

## 【問題5】平滑化  
平滑化するためのFlattenクラスを作成してください。


フォワードのときはチャンネル、高さ、幅の3次元を1次元にreshapeします。その値は記録しておき、バックワードのときに再びreshapeによって形を戻します。


この平滑化のクラスを挟むことで出力前の全結合層に適した配列を作ることができます。

class Flatten():
  def __init__(self):
    pass
  
  def forward(self, x):
    self.x = x
    N, C, H, W = self.x.shape
    x = x.reshape(N, -1)
    return x

  def backward(self, dout):
    N, C, H, W = self.x.shape
    dout = dout.reshape(N, C, H, W)
    return dout

In [ ]:
fla = Flatten()
a =fla.forward(pool.forward(xin))
display(a)
fla.backward(a)

# 3.検証

## 【問題6】学習と推定
作成したConv2dを使用してMNISTを学習・推定し、Accuracyを計算してください。


精度は低くともまずは動くことを目指してください。

In [ ]:
# メインクラス
class ScratchDeepNeuralNetworkClassifier():
  """
  送られてくるパラメーター
  sdnn = ScratchDeepNeuralNetworkClassifier(epoch_num=7)
  sdnn.fit(HeInitializer, ReLU, AdaGrad, x_train, y_train_one_hot, x_val, y_val_one_hot, sigma=0.01, lr=0.01)

  """

  def __init__(self, FN=3, n_output=10, epoch_num=1, batch_size=100, sigma=0.01, lr=0.01, verbose=True): 
    self.FN = FN
    self.n_output = n_output
    self.epoch = epoch_num
    self.batch_size = batch_size
    self.sigma = sigma
    self.lr = lr
    self.verbose = verbose
    self.num = 2
    self.loss = np.zeros(self.epoch)
    self.val_loss = np.zeros(self.epoch)

  def fit(self, initializing, act, optimization, X, y, X_val=None, y_val=None, n_features=784, n_nodes1=400, n_nodes2=200):  
    self.initializing0 = SimpleInitializer # CNN用の初期化クラスの設定
    self.initializing = initializing        # 初期化クラスの設定
    self.activation = act              # 活性化関数クラスの設定
    self.optimization = optimization      # 最適化クラスの設定
    self.h = None                    # 最適化クラス adagraidで使用
    self.x = x
    XN, XC, XH, XW = self.x.shape

    # 最適化クラスの設定
    optimizer0 = self.optimization(self.lr)
    optimizer1 = self.optimization(self.lr)
    optimizer2 = self.optimization(self.lr)
    optimizer3 = self.optimization(self.lr)

    # 畳み込みネットワークの設定
    self.CN1 = Conv2d(self.initializing0(self.sigma), optimizer0, XC, self.FN)
    self.activation1 = self.activation()
    # プーリング層の設定
    self.MaxPL1=Pooling()
    # 平滑化の設定
    self.FLAT1 = Flatten()
    # 全結合層にインスタンスを渡す
    self.FC1 = FC(n_features, n_nodes1, self.initializing(self.sigma), optimizer1)
    self.activation2 = self.activation()     
    self.FC2 = FC(n_nodes1, n_nodes2, self.initializing(self.sigma), optimizer2)
    self.activation３ = self.activation()        
    self.FC3 = FC(n_nodes2, self.n_output, self.initializing(self.sigma), optimizer3)
    self.activation4 = Softmax()

    # エポック数分の学習
    for i in range(self.epoch):
      # ミニバッチの作成
      get_mini_batch = GetMiniBatch(X, y, self.batch_size)
      # 1エポック（全バッチ）の学習
      for x_min, y_min in get_mini_batch:
        y_hat = self._forward_propagation(x_min)
        loss = self._back_propagation(X=y_hat, Y=y_min)
        # print("y_hat",y_hat[:1])

      self.loss[i] += loss
      if (type(X_val) != bool):
        self.val = 1
        y_hat_val = self._forward_propagation(X_val)
        loss_val = -np.mean(y_val_one_hot * np.log(y_hat_val + 1e-7))
        self.val_loss[i] += loss_val

      # self.acc_val[i] = accuracy_score(np.argmax(y_val, axis=1), np.argmax(y_hat_val, axis=1))
      # verboseをTrueにした際は学習過程を出力
      if self.verbose :
        print(f"--{i+1}回目~loss~-------\n{self.loss[i]}")
        print(f"--{i+1}回目~loss_val~---\n{self.val_loss[i]}")
        # print(f'epoch:{self.epoch:>3} loss:{self.loss:>8,.3f}')

  # フォワードプロパゲーションの実行
  def _forward_propagation(self, x):
    print("x",x)
    CN1 = self.CN1.forward(x)
    Z1 = self.activation1.forward(CN1)
    PL1 = self.MaxPL1.forward(Z1)
    FL1 =self.FLAT1.forward(PL1) 

    A1 = self.FC1.forward(FL1)
    Z2 = self.activation2.forward(A1)
    A2 = self.FC2.forward(Z1)
    Z3 = self.activation3.forward(A2)
    A3 = self.FC3.forward(Z2)
    Z4 = self.activation4.forward(A3)
    return Z4

  # バックプロパゲーションの実行
  def _back_propagation(self, X, Y):
    dA4, loss = self.activation4.backward(X, Y) # 交差エントロピー誤差とソフトマックスを合わせている
    dZ3 = self.FC3.backward(dA4)
    dA3 = self.activation3.backward(dZ3)
    dZ2 = self.FC2.backward(dA3)
    dA2 = self.activation2.backward(dZ2)
    dZ1 = self.FC1.backward(dA2)

    dFL1 = self.FLAT1.backward(dZ1)
    dPL1 = self.MaxPL1.backward(dFL1)
    dZ0 = self.activation1.backward(dPL1)
    dCN1 = self.CN1.backward(dZ0)
    return dCN1

  def predict(self, X):
    y_hat = self._forward_propagation(X)
    return np.argmax(y_hat, axis=1)

  def plot_cost(self):
    plt.title("Num_of_Iteration vs Loss")
    plt.xlabel("Num_of_Iteration")
    plt.ylabel("Loss")
    a = range(self.epoch)
    plt.plot(range(1, self.epoch+1), self.loss, color="b", label="train_loss")
    if self.val ==1:
        plt.plot(range(1, self.epoch+1), self.val_loss, color="orange", label="val_loss")
    plt.grid()
    plt.legend()

In [ ]:
x_train.shape

In [ ]:
sdnn = ScratchDeepNeuralNetworkClassifier(epoch_num=1, sigma=0.01, lr=0.01)
sdnn.fit(HeInitializer, ReLU, AdaGrad, x_train, y_train, x_val, y_val)

## layer

In [ ]:
# チャンネル数１の畳み込みネットワーク
class Conv2d():
  def __init__(self, initializer, optimizer, FC, FN, FH=3, FW=3, stride=1, padding=0):
    self.optimizer = optimizer
    self.w = initializer.W(FC, FN)
    self.b = initializer.B(FN)
    self.str = stride
    self.pad = padding
    self.col1 = im2col(filter_h=FH, filter_w=FW, stride=self.str, padding=self.pad)
    

    # 中間データ(backward時に使用)
    self.x = None
    self.col = None
    self.col_W = None

    # 重み・バイアスパラメーターの勾配
    self.dw = None
    self.db = None

  def forward(self, x):
    self.x = x
    FN, C, FH, FW = self.w.shape
    N, C, H, W = x.shape
    out_h, out_w = self.col1._output_size(H, W)
    
    # im2colクラスを定義し、実行する。
    self.col_im = self.col1.im2col(self.x)
    self.col_w = self.w.reshape(FN, -1).T
    # print('==========col===========\n', self.col_im)
    # print('=====================')
    # print('=========col_w===========\n', self.col_w)
    # print('=====================')

    out = (self.col_im @ self.col_w) + self.b
    # print('=========out1===========\n', out)
    # print('=====================')
    # reshapeで出力サイズを指定の形状に再構成、transposeで順番を入れ替え。
    out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
    # print('=========out2===========\n', out)
    # print('=====================')
    return out

  def backward(self, dout):
    FN, C, FH, FW = self.w.shape
    # print('========dout1===========\n', dout)
    # print('=====================')
    dout = dout.transpose(0,2,3,1).reshape(-1,FN)
    # print('========dout2===========\n', dout)
    # print('=====================')

    self.db = np.sum(dout, axis=0)
    # print('==========db===========\n', self.db)
    # print('=====================')
    self.dw = self.col_im.T @ dout
    # print('==========dw1===========\n', self.dw)
    # print('=====================')
    self.dw = self.dw.transpose(1, 0).reshape(FN, C, FH, FW)
    # print('=========dw2===========\n', self.dw)
    # print('=====================')
    dcol = dout @ self.col_w.T
    # print('========dcol===========\n', dcol)
    # print('=====================')
    # im2col状態を戻す処理
    dx = self.col.col2im(dcol, self.x.shape)
    # print('==========dx===========\n', dx)
    # print('=====================')
    
    # 更新
    self = self.optimizer.update(self)
    
    return self.dx

# Pooling層ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
class Pooling:
  def __init__(self, pool_h=2, pool_w=2, stride=2, padding=0):
    self.PH = pool_h
    self.PW = pool_w
    self.str = stride
    self.pad = padding
    # im2colクラスの定義
    self.col2 = im2col(filter_h=self.PH, filter_w=self.PW, stride=self.str, padding=self.pad)

    self.x = None
    self.arg_max = None

  def forward(self, x):
    # print('=========x===========\n', x.shape)
    # print('=====================')
    N, C, H, W = x.shape
    # out_h = int(1 + (H - self.pool_h) / self.stride)
    # out_w = int(1 + (W - self.pool_w) / self.stride)
    # im2colクラスのメソッドで、フィルターの上下、左右への移動数を求める。
    out_h, out_w = self.col2._output_size(H, W)
    # print('=========out_h===========\n', out_h)
    # print('=====================')
    # print('=========out_w===========\n', out_w)
    # print('=====================')

    # col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
    # col = col.reshape(-1, self.pool_h*self.pool_w)
    # 2次元配列に整形（2x2のフィルターで区切り、各カーネルの値を一列に並べフィルターを最後まで掛ける）
    self.col_im = self.col2.im2col(x)
    # print('=========col1===========\n', self.col_im)
    # print('=====================')
    # １行をPHxPW（４）列の配列に整形。
    col = self.col_im.reshape(-1, self.PH*self.PW)
    print('=========col2===========\n', col)
    print('=====================')

    # colの最大値のindexを返す
    arg_max = np.argmax(col, axis=1)
    print('=========max===========\n', arg_max)
    print('=====================')
    # indexから値を返す。
    out = np.max(col, axis=1)
    print('=========out1===========\n', out)
    print('=====================')
    # 配列順を変更し、次に渡したい型に整形。
    out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
    print('=========out2===========\n', out.shape)
    print('=====================')


    self.x = x
    self.arg_max = arg_max

    return out

  def backward(self, dout):
    # print('=========dout1===========\n', dout)
    # print('=====================')
    dout = dout.transpose(0, 2, 3, 1)
    # print('=========dout2===========\n', dout)
    # print('=====================')
    
    pool_size = self.PH * self.PW
    # print('=========pool_size===========\n', pool_size)
    # print('=====================')
    dmax = np.zeros((dout.size, pool_size))
    # print('=========dmax1===========\n', dmax)
    # print('=====================')
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
    # print('=========dmax2===========\n', dmax)
    # print('=====================')
    dmax = dmax.reshape(dout.shape + (pool_size,)) 
    # print('=========dmax3===========\n', dmax)
    # print('=====================')
    
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
    # print('=========dcol===========\n', dcol)
    # print('=====================')
    dx = self.col.col2im(dcol, self.x.shape)
    # print('=========dx===========\n', dx)
    # print('=====================')
    
    return dx

# 平滑化ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
class Flatten():
  def __init__(self):
    pass
  
  def forward(self, x):
    print('=======x===========\n', x.shape)
    print('=====================')
    self.x = x
    N, C, H, W = self.x.shape
    x = x.reshape(N, -1)
    return x

  def backward(self, dout):
    N, C, H, W = self.x.shape
    dout = dout.reshape(N, C, H, W)
    return dout

# 全結合層ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = initializer.B(self.n_nodes2)
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する

    # フォワードプロパゲーション時の処理
    def forward(self, X):
      print('=======X===========\n', X.shape)
      print('=====================')
      print('========W=========\n', self.W.shape)
      print('=====================')
      self.X = X
      print("W", )
      out = X@self.W+self.B
      return out
    
    # バックプロパゲーション時の処理
    def backward(self, dA):
      self.dZ = dA@(self.W.T)
      self.dW = (self.X.T)@dA
      self.dB = np.sum(dA, axis=0)
      
      # 更新
      self = self.optimizer.update(self)
      return self.dZ

In [ ]:
# 初期化クラス
# SimpleInitializer ------------------------------------------------------------
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, FC, FN, FH=3, FW=3):
        W = self.sigma * np.random.randn(FN, FC, FH, FW)
        return W

    def B(self, FN):
      B = self.sigma * np.random.randn(1, FN)
      return B

# XavierInitializer ------------------------------------------------------------
class XavierInitializer:
    def __init__(self, sigma):
      self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        W = np.random.randn(n_nodes1, n_nodes2)/np.sqrt(n_nodes1)
        return W

    def B(self, n_nodes2):
        return np.zeros(n_nodes2)

# He-----------------------------------------------------------------------------
class HeInitializer:
    def __init__(self, sigma):
      self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(2/n_nodes1) 
        return W

    def B(self, n_nodes2):
        return np.zeros(n_nodes2)

In [ ]:
# 最適化クラス
# SGD---------------------------------------------
class SGD:
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
      layer.W -= self.lr * layer.dW
      layer.B -= self.lr * layer.dB
      return layer

# AdaGrad----------------------------------------
class AdaGrad:
    def __init__(self, lr):
      self.lr = lr
      self.hw = 0
      self.hb = 0
  
    def update(self, layer):
        self.hw += layer.dW * layer.dW
        self.hb = layer.dB * layer.dB
        layer.W -= self.lr * layer.dW / (np.sqrt(self.hw) +1e-7)
        layer.B -= self.lr * layer.dB / (np.sqrt(self.hb) +1e-7)
        return layer

In [ ]:
# 活性化関数クラス
# ソフトマックス関数のクラス----------------------------------------------
class Softmax:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    y_hat = np.exp(X) / np.sum(np.exp(X), axis = 1).reshape(-1,1)
    return y_hat

  # backward時の処理
  def backward(self, X, Y):
    loss = -np.mean(Y * np.log(X + 1e-7))
    dA3 = X - Y
    return dA3, loss

# ReLU関数のクラス----------------------------------------------
class ReLU:
  def __init__(self):
        pass
  # forward時の処理
  def forward(self, X):
    self.X =X
    return np.maximum(0, X)
    
  # backward時の処理
  def backward(self, dout):
    return np.where(self.X > 0, dout, 0)

# tanh関数のクラス----------------------------------------------
class Tanh:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, X):
    self.out = np.tanh(X)
    return self.out

  # backward時の処理
  def backward(self, X):
    return X*(1-self.out**2)

# sigmoid関数のクラス----------------------------------------------
class Sigmoid:
  def __init__(self):
    pass

  # forward時の処理
  def forward(self, z):
    self.out = 1 / (1+np.exp(-z))
    return self.out

  # backward時の処理
  def backward(self, z):
    return z*(1-self.out)*self.out

### 前処理

In [ ]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# # 平滑化（flatten）
# X_train_fltten = X_train.reshape(-1, 784)
# X_test_fltten = X_test.reshape(-1, 784)

# float化と0or1処理
X_train_flt = X_train.astype(np.float)
X_test_flt = X_test.astype(np.float)
X_train_flt /= 255
X_test_flt /= 255

X_train = X_train_flt[:, np.newaxis,:,:]
X_test = X_test_flt[:, np.newaxis,:,:]

# one hot処理
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

In [ ]:
# train, testの分割
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(np.array(X_train), np.array(y_train_one_hot), test_size=0.2)
print("x_train",x_train.shape, "x_val", x_val.shape, "y_train.shape", y_train.shape, "y_val,shape", y_val.shape) # (48000, 784)

In [ ]:
y_pred = sdnn.predict(X_test_flt)
y_pred[:100]

In [ ]:
y_test[:100]

In [ ]:
sdnn.plot_cost()